In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126947")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126947
Azure region: southcentralus
Subscription id: fb968fd6-afa4-4fb2-8296-1d0120d715b4
Resource group: aml-quickstarts-126947


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Reference: lesson 5.9 and Microsoft website at:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling( 
    {
    '--C': choice(0.0001, 0.001, 0.01, 0.1, 1,10,100,1000),
    '--max_iter': choice(100, 200, 300, 400, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"    
    
# Reference: lesson 6.3: copying the training file into the script folder
shutil.copy('./train.py', script_folder)
    

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = script_folder, 
              compute_target=cpu_cluster, 
              entry_script="train.py")



In [5]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                          hyperparameter_sampling=ps,
                          policy=policy, 
                          primary_metric_name="Accuracy", 
                          primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs = 15)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config = hyperdrive_config)

#reference: lesson 6.3: running a history widget to show the progress
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_61298a5c-3264-4eb1-9916-12d78718f232
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_61298a5c-3264-4eb1-9916-12d78718f232?wsid=/subscriptions/fb968fd6-afa4-4fb2-8296-1d0120d715b4/resourcegroups/aml-quickstarts-126947/workspaces/quick-starts-ws-126947

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-19T21:01:14.392228][API][INFO]Experiment created<END>\n""<START>[2020-11-19T21:01:14.901779][GENERATOR][INFO]Trying to sample '15' jobs from the hyperparameter space<END>\n"<START>[2020-11-19T21:01:15.5692760Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-11-19T21:01:15.380267][GENERATOR][INFO]Successfully sampled '15' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_61298a5c-3264-4eb1-9916-12d78718f232
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_61298a5c-3264-4eb1-9916-12d78718f232?wsid=/subscriptions/fb9

In [7]:
# Get your best run and save the model from that run.
#reference lesson 6.3: finding and registering best model

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print("Parameters of the best run: ",parameter_values)
print('Best Run Id: ', best_run.id)
print( "Accuracy:", best_run_metrics['Accuracy'])



Parameters of the best run:  ['--C', '0.1', '--max_iter', '500']
Best Run Id:  HD_61298a5c-3264-4eb1-9916-12d78718f232_1
Accuracy: 0.9081436519979768


In [8]:
import joblib 

# Save the model
best_run.download_file("/outputs/model.joblib", "Logistic_reg_hd_bankmarketing.joblib")
#model = best_run.register_model(model_name = 'Logistic_reg_hd_bankmarketing', model_path = 'outputs/model.joblib')

In [9]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_4ff4eb404373703fe2b6097b7dc6e950af789061bb7b3bd01a58fdfa4aa69a03_d.txt',
 'azureml-logs/65_job_prep-tvmps_4ff4eb404373703fe2b6097b7dc6e950af789061bb7b3bd01a58fdfa4aa69a03_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_4ff4eb404373703fe2b6097b7dc6e950af789061bb7b3bd01a58fdfa4aa69a03_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/101_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [12]:
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [13]:
import pandas as pd

# Because we are doing k-fold cross validation 
# and the validation dataset is not mentioned as a parameter of AutoMLConfig we need to set
#in the assignment, there is no need to split the data into the train and test sets.

# The training_data parameter should contain both training features and a label column 
training_data = pd.concat([x, y], axis = 1)

In [14]:
training_data.head()


,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [15]:
training_data.describe()


,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,...,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,0.605948,0.000091,0.522974,0.151806,6.605281,2.980789,257.335205,2.561730,962.174780,...,0.364310,0.101153,0.056055,0.147496,0.229226,0.000455,0.128346,0.294901,0.042367,0.112049
std,10.432313,0.488653,0.009542,0.499479,0.358838,2.041099,1.411580,257.331700,2.763646,187.646785,...,0.481243,0.301536,0.230031,0.354605,0.420341,0.021332,0.334480,0.456005,0.201429,0.315431
min,17.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,102.000000,1.000000,999.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,38.000000,1.000000,0.000000,1.000000,0.000000,6.000000,3.000000,179.000000,2.000000,999.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,47.000000,1.000000,0.000000,1.000000,0.000000,8.000000,4.000000,318.000000,3.000000,999.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,98.000000,1.000000,1.000000,1.000000,1.000000,12.000000,5.000000,4918.000000,56.000000,999.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
training_data[training_data.isnull().any(axis=1)]


,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y


In [19]:
training_data["y"].value_counts()

0    29258
1     3692
Name: y, dtype: int64

In [20]:
#Converting Pandas DataFrame to a supported type for the training_data parameter in AutoMLConfig

#Supported types: [azureml.data.tabular_dataset.TabularDataset, 
#azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]"

#There are two methods to convert the dataframe into a supported type: 
# 1. Using register_pandas_dataframe method
# 2. Converting DataFrame to CSV and then to azureml.data.tabular_dataset.TabularDataset type
# The first one is experimental and may change at any time, therefore, we should use the other method.

# Reference: "Create a dataset from pandas dataframe" section 
# at https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets

from azureml.core import Workspace, Dataset

#creating a folder
#dirname = "./training_data"
os.makedirs('training_data', exist_ok=True)

local_path = './training_data/training_data.csv'
training_data.to_csv(local_path)

# upload the local file to a datastore on the cloud
workspace = Workspace(ws.subscription_id, ws.resource_group, ws.name)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='training_data', target_path='training_data')

# create a dataset referencing the cloud location
training_data = Dataset.Tabular.from_delimited_files(path = [(datastore, ('training_data/training_data.csv'))])




Uploading an estimated of 1 files
Uploading training_data/training_data.csv
Uploaded training_data/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name = "y",
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [22]:
# Submit your automl run

exp = Experiment(workspace=ws, name="AutoML-project")
automl_run = exp.submit(config = automl_config)

# Reference: lesson 6.3: running a history widget to show the progress
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")


Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=====

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [23]:
# Retrieve and save your best automl model.
# Reference: https://knowledge.udacity.com/questions/357775
automl_best_run, fitted_automl_model = automl_run.get_output() 
print("Fitted model: ", fitted_automl_model.steps[-1])
print("---"*5)
print("AutoML best run accuracy: ", automl_best_run.get_metrics(name="accuracy"))
print("---"*5)
print("AutoML run Summary: ", automl_run.summary())

Fitted model:  ('stackensembleclassifier', StackEnsembleClassifier(base_learners=[('1',
                                        Pipeline(memory=None,
                                                 steps=[('maxabsscaler',
                                                         MaxAbsScaler(copy=True)),
                                                        ('xgboostclassifier',
                                                         XGBoostClassifier(base_score=0.5,
                                                                           booster='gbtree',
                                                                           colsample_bylevel=1,
                                                                           colsample_bynode=1,
                                                                           colsample_bytree=1,
                                                                           gamma=0,
                                                              

In [50]:
estimator_automl = fitted_automl_model.steps[-1]
with open("./project.txt", 'w') as output:
    output.write(str(estimator_automl))


In [24]:
# Reference: https://knowledge.udacity.com/questions/384083 Mentor: Ahmet Can S

#Save the model
automl_best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')


Model(workspace=Workspace.create(name='quick-starts-ws-126947', subscription_id='fb968fd6-afa4-4fb2-8296-1d0120d715b4', resource_group='aml-quickstarts-126947'), name=automl_best_model.pkl, id=automl_best_model.pkl:1, version=1, tags={}, properties={})

In [74]:
cpu_cluster.delete()
